In [4]:
import gym
import numpy as np
import random
import datetime
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt
# todo : CartPole-v0


"""
#! ghi chú về bài toán

    *Description:
        The agent (a car) is started at the bottom of a valley. For any given
        state the agent may choose to accelerate to the left, right or cease
        any acceleration.
    *Source:
        The environment appeared first in Andrew Moore's PhD Thesis (1990).
    *Observation:
        Type: Box(2)
        Num    Observation               Min            Max
        0      Car Position              -1.2           0.6
        1      Car Velocity              -0.07          0.07
    *Actions:
        Type: Discrete(3)
        Num    Action
        0      Accelerate to the Left
        1      Don't accelerate
        2      Accelerate to the Right
        Note: This does not affect the amount of velocity affected by the
        gravitational pull acting on the car.
    *Reward:
         Reward of 0 is awarded if the agent reached the flag (position = 0.5)
         on top of the mountain.
         Reward of -1 is awarded if the position of the agent is less than 0.5.
    *Starting State:
         The position of the car is assigned a uniform random value in
         [-0.6 , -0.4].
         The starting velocity of the car is always assigned to 0.
    *Episode Termination:
         The car position is more than 0.5
         Episode length is greater than 200

"""

In [8]:
def get_discrete_state(state):
    discrete_state = (state - env.observation_space.low)/discrete_os_win_size
    return tuple(discrete_state.astype(np.int64))

In [38]:
# Q-Learning settings
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 25000

# Exploration settings
epsilon = 1  # not a constant, qoing to be decayed
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//2
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)

print(END_EPSILON_DECAYING)
print(epsilon_decay_value)

SHOW_EVERY = 1000


12500
8.000640051204096e-05


In [28]:
env = gym.make("MountainCar-v0")
env.reset()

DISCRETE_OS_SIZE = [20, 20]
discrete_os_win_size = (env.observation_space.high - env.observation_space.low)/DISCRETE_OS_SIZE

q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))
print('q_table',q_table.shape)

new_state = env.reset()
discrete_state = get_discrete_state(new_state)


for episode in range(EPISODES):
    discrete_state = get_discrete_state(env.reset())
    done = False

    if episode % SHOW_EVERY == 0:
        render = True
        #print(episode)
    else:
        render = False

    while not done:

        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(q_table[discrete_state])
        else:
            # Get random action
            action = np.random.randint(0, env.action_space.n)


        new_state, reward, done, _ = env.step(action)

        new_discrete_state = get_discrete_state(new_state)

        if episode % SHOW_EVERY == 0:
            env.render()
        #new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

        # If simulation did not end yet after last step - update Q table
        if not done:

            # Maximum possible Q value in next step (for new state)
            max_future_q = np.max(q_table[new_discrete_state])

            # Current Q value (for current state and performed action)
            current_q = q_table[discrete_state + (action,)]

            # And here's our equation for a new Q value for current state and action
            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

            # Update Q table with new Q value
            q_table[discrete_state + (action,)] = new_q


        # Simulation ended (for any reson) - if goal position is achived - update Q value with reward directly
        elif new_state[0] >= env.goal_position:
            #q_table[discrete_state + (action,)] = reward
            q_table[discrete_state + (action,)] = 0

        discrete_state = new_discrete_state

    # Decaying is being done every episode if episode number is within decaying range
    if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
        print('END_EPSILON_DECAYING',END_EPSILON_DECAYING,'episode',episode,'START_EPSILON_DECAYING',START_EPSILON_DECAYING)

        epsilon -= epsilon_decay_value


env.close()




q_table (20, 20, 3)


Exception ignored in: <function Viewer.__del__ at 0x000002628C416430>
Traceback (most recent call last):
  File "C:\Anaconda\envs\ai_reinforcement_learning\lib\site-packages\gym\envs\classic_control\rendering.py", line 185, in __del__
    self.close()
  File "C:\Anaconda\envs\ai_reinforcement_learning\lib\site-packages\gym\envs\classic_control\rendering.py", line 101, in close
    self.window.close()
  File "C:\Anaconda\envs\ai_reinforcement_learning\lib\site-packages\pyglet\window\win32\__init__.py", line 328, in close
    super(Win32Window, self).close()
  File "C:\Anaconda\envs\ai_reinforcement_learning\lib\site-packages\pyglet\window\__init__.py", line 857, in close
    app.windows.remove(self)
  File "C:\Anaconda\envs\ai_reinforcement_learning\lib\_weakrefset.py", line 110, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x000002628C4144F0; to 'Win32Window' at 0x000002628B5FF880>


END_EPSILON_DECAYING 12500 episode 1 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 2 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 3 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 4 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 5 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 6 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 7 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 8 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 9 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 10 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 11 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 12 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 13 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 14 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 15 START_EPSILON_DECAYING 1
END_EPSILON_DECAYING 12500 episode 16 START_EPSILON_DECAYING 1
E

KeyboardInterrupt: 

In [30]:
def save_weight(dirfile,weight):
    return np.save(dirfile,weight)

def load_weight(dirfile):
    return np.load(dirfile)


In [31]:
dirfile = 'weight\MountainCar-v0_Qtable_v001.npy'

save_weight(dirfile,q_table)

In [36]:
load_q = load_weight(dirfile)




for i in range(10):
    discrete_state = get_discrete_state(env.reset())
    done = False
    while not done:
        env.render()

        action = np.argmax(load_q[discrete_state])
        new_state, reward, done, _ = env.step(action)

        discrete_state = get_discrete_state(new_state)

    env.close()    
